In [1]:
import pandas as pd
import numpy as np

In [19]:
from keras.models import Sequential
from keras.layers import Dense

2023-01-27 11:37:25.441711: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 11:37:25.611452: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-27 11:37:25.611485: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-27 11:37:26.407790: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
train_data = pd.read_csv("../data_acquisition/data_0.3/data_game_values_train.csv", sep=";")

In [4]:
xg_mean = np.mean(np.mean(train_data["home_xG"]) + np.mean(train_data["away_xG"]))
xg_against_mean = np.mean(np.mean(train_data["home_xg_against"]) + np.mean(train_data["away_xg_against"]))

xt_mean = np.mean(np.mean(train_data["home_xT_all"]) + np.mean(train_data["away_xT_all"]))
xt_against_mean = np.mean(np.mean(train_data["home_xt_all_against"]) + np.mean(train_data["away_xt_all_against"]))

form_for_mean = np.mean(np.mean(train_data["ha_form_home_for"] / 5) + np.mean(train_data["ha_form_away_for"] / 5))
form_against_mean = np.mean(np.mean(train_data["ha_form_home_against"] / 5) + np.mean(train_data["ha_form_away_against"] / 5))

In [72]:
features = np.swapaxes(np.array([train_data["home_xG"] - xg_mean, train_data["away_xg_against"] - xg_against_mean,
                                train_data["away_xG"] - xg_mean, train_data["home_xg_against"] - xg_against_mean,
                                train_data["home_xT_all"] - xt_mean, train_data["away_xt_all_against"] - xt_against_mean,
                                train_data["away_xT_all"] - xt_mean, train_data["home_xt_all_against"] - xt_against_mean]), 0, 1)

# gi = np.swapaxes(np.array([train_data["home_gi"], train_data["away_gi"]]), 0, 1)

form = np.swapaxes(np.array([(train_data["ha_form_home_for"] / 5) - form_for_mean, (train_data["ha_form_home_against"] / 5) - form_against_mean,
                            (train_data["ha_form_away_for"] / 5) - form_for_mean, (train_data["ha_form_away_against"] / 5) - form_against_mean]), 0, 1)

goals = np.swapaxes(np.array([train_data["home_score"], train_data["away_score"]]), 0, 1)

elo_diff = np.swapaxes(np.array([(train_data["elo_home"] / 1000) - (train_data["elo_away"] / 1000)]), 0, 1)
       

In [80]:
goals_clas = [np.array([1, 0, 0]) if x[0] > x[1] else np.array([0, 1, 0]) if x[0] == x[1] else np.array([0, 0, 1]) for x in goals]

In [81]:
X = np.concatenate([features, form, elo_diff], axis=1)
y = np.array(goals_clas)

In [83]:
model = Sequential()
model.add(Dense(26, input_shape=(13,), activation='relu'))
model.add(Dense(52, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(3, activation="sigmoid"))

In [84]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mean_squared_error'])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

In [85]:
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)

Epoch 1/150
609/609 [==============================] - 2s 2ms/step - loss: 0.5397 - mean_squared_error: 0.1811
Epoch 2/150
609/609 [==============================] - 1s 2ms/step - loss: 0.5169 - mean_squared_error: 0.1724
Epoch 3/150
609/609 [==============================] - 1s 2ms/step - loss: 0.5137 - mean_squared_error: 0.1713
Epoch 4/150
609/609 [==============================] - 1s 2ms/step - loss: 0.5123 - mean_squared_error: 0.1708
Epoch 5/150
609/609 [==============================] - 1s 2ms/step - loss: 0.5115 - mean_squared_error: 0.1705
Epoch 6/150
609/609 [==============================] - 1s 2ms/step - loss: 0.5106 - mean_squared_error: 0.1700
Epoch 7/150
609/609 [==============================] - 1s 2ms/step - loss: 0.5098 - mean_squared_error: 0.1698
Epoch 8/150
609/609 [==============================] - 1s 2ms/step - loss: 0.5103 - mean_squared_error: 0.1700
Epoch 9/150
609/609 [==============================] - 1s 2ms/step - loss: 0.5098 - mean_squared_error: 0.1699
E

In [86]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

191/191 [==============================] - 1s 2ms/step - loss: 0.4509 - mean_squared_error: 0.1489
Accuracy: 14.89


In [87]:
test_data = pd.read_csv("../data_acquisition/data_0.3/data_game_values_test_odds.csv", sep=";").dropna()

features = np.swapaxes(np.array([test_data["home_xG"] - xg_mean, test_data["away_xg_against"] - xg_against_mean,
                                test_data["away_xG"] - xg_mean, test_data["home_xg_against"] - xg_against_mean,
                                test_data["home_xT_all"] - xt_mean, test_data["away_xT_all"] - xt_mean,
                                test_data["home_xt_all_against"] - xt_against_mean, test_data["away_xt_all_against"] - xt_against_mean]), 0, 1)
# features_a = np.swapaxes(np.array([train_data["away_xG"], train_data["away_xT_only_pos"], train_data["home_xg_against"], train_data["home_xt_only_pos_against"]]), 0, 1)

gi = np.swapaxes(np.array([test_data["home_gi"], test_data["away_gi"]]), 0, 1)
# gi_a = np.swapaxes(np.array([train_data["away_gi"]]), 0, 1)

form = np.swapaxes(np.array([(test_data["ha_form_home_for"] / 5) - form_for_mean, (test_data["ha_form_home_against"] / 5) - form_against_mean,
                            (test_data["ha_form_away_for"] / 5) - form_for_mean, (test_data["ha_form_away_against"] / 5) - form_against_mean]), 0, 1)

goals = np.swapaxes(np.array([test_data["home_score"], test_data["away_score"]]), 0, 1)

elo = np.swapaxes(np.array([(test_data["elo_home"] / 1000) - (test_data["elo_away"] / 1000)]), 0, 1)


In [88]:
goals_clas = [np.array([1, 0, 0]) if x[0] > x[1] else np.array([0, 1, 0]) if x[0] == x[1] else np.array([0, 0, 1]) for x in goals]

In [89]:
X_test = np.concatenate([features, form, elo], axis=1)
y_test = np.array(goals_clas)

In [90]:
predictions = model.predict(X_test)

49/49 [==============================] - 0s 2ms/step


In [91]:
# pred, act = [], []
# for idx, i in enumerate(np.round(predictions)):
#     if i[0] > i[1]:
#         pred.append(0)
#     if i[0] == i[1]:
#         pred.append(1)
#     if i[0] < i[1]:
#         pred.append(2)
#     if y_test[idx][0] > y_test[idx][1]:
#         act.append(0)
#     if y_test[idx][0] == y_test[idx][1]:
#         act.append(1)
#     if y_test[idx][0] < y_test[idx][1]:
#         act.append(2)
    
# df = pd.DataFrame({"act": act, "pred": pred})

In [92]:
pred, act = [], []
for idx, i in enumerate(np.round(predictions)):
    pred.append(np.argmax(i))
    if y_test[idx][0] > y_test[idx][1]:
        act.append(0)
    if y_test[idx][0] == y_test[idx][1]:
        act.append(1)
    if y_test[idx][0] < y_test[idx][1]:
        act.append(2)
    
df = pd.DataFrame({"act": act, "pred": pred})

In [93]:
np.sum(df[df["act"] == df["pred"]]) / df.shape[0]

act     0.144337
pred    0.144337
dtype: float64

In [94]:
df

,act,pred
0,2,0
1,0,1
2,0,0
3,0,0
4,1,0
...,...,...
1540,1,2
1541,0,0
1542,1,1
1543,2,0
